## Lab 4 Adversarial Learning
### Name: Lakhan Kumar Sunilkumar
### SID: 862481700
### Email: lsuni001@ucr.edu

In [1]:
# Imports, CUDA check, and safe globals for ResNet

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.serialization import add_safe_globals
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import math

# Register all classes used inside torchvision ResNet as safe
add_safe_globals([
    ResNet,
    BasicBlock,
    Bottleneck,
    nn.Conv2d,
    nn.BatchNorm2d,
    nn.Linear,
    nn.ReLU,
    nn.MaxPool2d,
    nn.AdaptiveAvgPool2d,
    nn.Sequential,
    nn.Dropout,
])

# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())

for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"CUDA device {i}: {props.name}")

print("Using device:", device)


PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device count: 1
CUDA device 0: NVIDIA GeForce RTX 3060 Laptop GPU
Using device: cuda:0


In [2]:
# Load pre-trained ResNet model safely 

# Path to model file
RESNET_MODEL_FILE = "./resnet.model"

# Load the model with safe globals
model = torch.load(
    RESNET_MODEL_FILE,
    map_location=device,
    weights_only=True
)

model = model.to(device)
model.eval()

print("Model loaded with weights_only=True and set to eval() on", device)
print("Model type:", type(model))


Model loaded with weights_only=True and set to eval() on cuda:0
Model type: <class 'torchvision.models.resnet.ResNet'>


In [3]:
TESTSET_FILE = "./testset.pt"

testset_obj = torch.load(TESTSET_FILE, map_location="cpu", weights_only=True)
print("Loaded testset object type:", type(testset_obj))

# Case 1: (input_imgs, labels) tuple of tensors
if (
    isinstance(testset_obj, (tuple, list))
    and len(testset_obj) == 2
    and torch.is_tensor(testset_obj[0])
    and torch.is_tensor(testset_obj[1])
):
    input_imgs, labels = testset_obj
    print("Interpreting testset as (input_imgs, labels) tensors")
    print("input_imgs shape:", input_imgs.shape)
    print("labels shape:", labels.shape)
    dataset = TensorDataset(input_imgs, labels)

# Case 2: list of (img, label) pairs, labels as ints
elif (
    isinstance(testset_obj, list)
    and len(testset_obj) > 0
    and isinstance(testset_obj[0], tuple)
    and len(testset_obj[0]) == 2
):
    print("Interpreting testset as list of (img, label) pairs")
    dataset = testset_obj
    input_imgs = torch.stack([x for x, _ in dataset], dim=0)
    labels = torch.tensor([int(y) for _, y in dataset], dtype=torch.long)
    print("Derived stacked input_imgs shape:", input_imgs.shape)
    print("Derived stacked labels shape:", labels.shape)
else:
    raise RuntimeError("Unexpected format of testset.pt")

NUM_SAMPLES = input_imgs.shape[0]
print("Number of samples:", NUM_SAMPLES)

BATCH_SIZE = 8 

testloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
)

print("DataLoader ready; number of batches:", len(testloader))


Loaded testset object type: <class 'list'>
Interpreting testset as list of (img, label) pairs
Derived stacked input_imgs shape: torch.Size([100, 3, 32, 32])
Derived stacked labels shape: torch.Size([100])
Number of samples: 100
DataLoader ready; number of batches: 13


In [4]:
# Peek at a batch to infer value range
sample_imgs, _ = next(iter(testloader))
vmin = sample_imgs.min().item()
vmax = sample_imgs.max().item()
print(f"Sample value range: [{vmin:.3f}, {vmax:.3f}]")

if vmin >= -1.05 and vmax <= 1.05:
    CLAMP_MIN, CLAMP_MAX = -1.0, 1.0
    print("Detected normalized range ~[-1, 1]; using clamp [-1, 1]")
elif vmin >= -0.01 and vmax <= 1.01:
    CLAMP_MIN, CLAMP_MAX = 0.0, 1.0
    print("Detected range ~[0, 1]; using clamp [0, 1]")
else:
    # fallback: use observed min/max with small buffer
    CLAMP_MIN, CLAMP_MAX = vmin - 1e-3, vmax + 1e-3
    print(f"Using clamp [{CLAMP_MIN:.3f}, {CLAMP_MAX:.3f}] (fallback)")


Sample value range: [-1.000, 1.000]
Detected normalized range ~[-1, 1]; using clamp [-1, 1]


In [5]:
# Compute Top-1 / Top-5 accuracy

import torch.nn.functional as F

@torch.no_grad()
def compute_topk_accuracies(model, loader, device, ks=(1, 5)):
    model.eval()
    correct_k = {k: 0 for k in ks}
    total = 0

    for imgs, labels in tqdm.tqdm(loader, desc="Evaluating (clean)..."):
        imgs = imgs.to(device)
        labels = labels.to(device)

        logits = model(imgs) # (N, 1000)
        probs = F.softmax(logits, dim=1)

        total += labels.size(0)

        for k in ks:
            topk_idx = probs.topk(k, dim=1).indices # (N, k)
            match = topk_idx.eq(labels.view(-1, 1)).any(dim=1)
            correct_k[k] += match.sum().item()

    topk_acc = {k: correct_k[k] / total for k in ks}
    return topk_acc

# Baseline accuracy on original test images

orig_topk = compute_topk_accuracies(model, testloader, device, ks=(1, 5))
print(f"Original Top-1 accuracy: {orig_topk[1]:.4f}")
print(f"Original Top-5 accuracy: {orig_topk[5]:.4f}")


Evaluating (clean)...: 100%|██████████| 13/13 [00:01<00:00, 11.39it/s]

Original Top-1 accuracy: 0.8200
Original Top-5 accuracy: 0.9700


In [6]:
def fgsm_attack(model, input_imgs, labels, epsilon):
    """
    Standard untargeted FGSM:
      x_adv = clamp(x + eps * sign(grad_x L(model(x), y)), [CLAMP_MIN, CLAMP_MAX])
    """
    model.eval()

    input_imgs = input_imgs.to(device)
    labels = labels.to(device)

    input_imgs.requires_grad = True

    preds = model(input_imgs)
    loss  = nn.CrossEntropyLoss()(preds, labels)

    model.zero_grad()
    loss.backward()

    grad = input_imgs.grad

    adv = input_imgs + epsilon * torch.sign(grad)
    adv = torch.clamp(adv, CLAMP_MIN, CLAMP_MAX)

    return adv.detach()


In [7]:
def pgd_attack(model, input_imgs, labels, epsilon, alpha, num_iter):
    """
    L_inf PGD with projection onto epsilon-ball around original x.
    """
    model.eval()

    input_imgs = input_imgs.to(device)
    labels     = labels.to(device)

    # Start exactly from original images (no random start to keep eps tight)
    x = input_imgs.clone().detach()
    x.requires_grad = True

    for _ in tqdm.tqdm(range(num_iter), desc="PGD iters", leave=False):
        preds = model(x)
        loss  = nn.CrossEntropyLoss()(preds, labels)

        model.zero_grad()
        loss.backward()

        grad = x.grad

        # Gradient ascent on loss
        x = x + alpha * torch.sign(grad)

        # Project back to L_inf ball
        x = torch.max(torch.min(x, input_imgs + epsilon), input_imgs - epsilon)

        # Clamp to valid input range
        x = torch.clamp(x, CLAMP_MIN, CLAMP_MAX)

        x = x.detach()
        x.requires_grad = True

    return x.detach()


In [8]:
def build_adv_dataset_fgsm(model, loader, epsilon):
    adv_imgs = []
    adv_labels = []

    for imgs, lbls in tqdm.tqdm(loader, desc=f"FGSM eps={epsilon:.4f}"):
        adv_batch = fgsm_attack(model, imgs, lbls, epsilon=epsilon)
        for img_adv, lbl in zip(adv_batch, lbls):
            adv_imgs.append(img_adv.cpu())
            adv_labels.append(int(lbl))

    return adv_imgs, torch.tensor(adv_labels, dtype=torch.long)

def build_adv_dataset_pgd(model, loader, epsilon, alpha, num_iter):
    adv_imgs = []
    adv_labels = []

    for imgs, lbls in tqdm.tqdm(loader, desc=f"PGD eps={epsilon:.4f}, iters={num_iter}"):
        adv_batch = pgd_attack(model, imgs, lbls, epsilon=epsilon, alpha=alpha, num_iter=num_iter)
        for img_adv, lbl in zip(adv_batch, lbls):
            adv_imgs.append(img_adv.cpu())
            adv_labels.append(int(lbl))

    return adv_imgs, torch.tensor(adv_labels, dtype=torch.long)

def compute_adv_topk(model, adv_imgs, adv_labels, batch_size=8):
    adv_dataset = list(zip(adv_imgs, adv_labels))
    adv_loader = DataLoader(adv_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return compute_topk_accuracies(model, adv_loader, device, ks=(1,5))

def compute_empirical_eps(orig_imgs, adv_imgs):
    orig = orig_imgs
    adv = torch.stack(adv_imgs, dim=0)
    assert orig.shape == adv.shape
    diff = (adv - orig).abs()
    return diff.mean().item()


In [9]:
def estimate_fgsm_score(tp1_diff, tp5_diff, eps_empirical):
    
    tp1_score = 0.33 + 33.0 * tp1_diff
    tp5_score = 66.0 * tp5_diff
    fgsm_score = max(tp1_score, tp5_score)
    
    fgsm_score = max(0.0, min(fgsm_score, 10.0))
    return fgsm_score, tp1_score, tp5_score

def estimate_pgd_score(tp1_diff, tp5_diff, eps_empirical):
    
    tp1_score = 26.0 * tp1_diff
    tp5_score = 46.0 * tp5_diff
    inner = max(tp1_score, tp5_score)
    inner = max(0.0, min(inner, 10.0))
    pgd_bonus = 0.2 * inner
    return pgd_bonus, tp1_score, tp5_score


In [10]:

fgsm_eps_grid = [0.05, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]
EPS_THRESHOLD = 0.20  

best_fgsm_eps = None
best_fgsm_adv_top1 = None
best_fgsm_adv_top5 = None
best_fgsm_imgs = None
best_fgsm_labels = None
best_fgsm_score = -1.0
best_fgsm_eps_emp = None

for eps in fgsm_eps_grid:
    adv_imgs_fgsm, adv_lbls_fgsm = build_adv_dataset_fgsm(model, testloader, epsilon=eps)
    adv_topk = compute_adv_topk(model, adv_imgs_fgsm, adv_lbls_fgsm, batch_size=BATCH_SIZE)

    tp1_diff = abs(orig_topk[1] - adv_topk[1])
    tp5_diff = abs(orig_topk[5] - adv_topk[5])
    eps_emp  = compute_empirical_eps(input_imgs, adv_imgs_fgsm)

    
    fgsm_score, tp1_s, tp5_s = estimate_fgsm_score(tp1_diff, tp5_diff, eps_emp)

    print(
        f"[FGSM eps={eps:.3f}] "
        f"Top1_adv={adv_topk[1]:.3f}, Top5_adv={adv_topk[5]:.3f}, "
        f"tp1_diff={tp1_diff:.3f}, tp5_diff={tp5_diff:.3f}, eps_emp={eps_emp:.3f}, "
        f"tp1_score={tp1_s:.3f}, tp5_score={tp5_s:.3f}, est_score={fgsm_score:.3f}"
    )

    
    if eps_emp > EPS_THRESHOLD:
        print(f"  -> Skipping eps={eps:.3f} because eps_emp={eps_emp:.3f} > {EPS_THRESHOLD}")
        continue

    
    if fgsm_score > best_fgsm_score:
        best_fgsm_score = fgsm_score
        best_fgsm_eps = eps
        best_fgsm_adv_top1 = adv_topk[1]
        best_fgsm_adv_top5 = adv_topk[5]
        best_fgsm_imgs = adv_imgs_fgsm
        best_fgsm_labels = adv_lbls_fgsm
        best_fgsm_eps_emp = eps_emp

# Fallback: if nothing satisfied eps_emp <= 0.2, pick the smallest eps
if best_fgsm_eps is None:
    print("\nNo FGSM candidate with eps_emp <= 0.2; falling back to smallest eps in grid.")
    eps = min(fgsm_eps_grid)
    adv_imgs_fgsm, adv_lbls_fgsm = build_adv_dataset_fgsm(model, testloader, epsilon=eps)
    adv_topk = compute_adv_topk(model, adv_imgs_fgsm, adv_lbls_fgsm, batch_size=BATCH_SIZE)
    tp1_diff = abs(orig_topk[1] - adv_topk[1])
    tp5_diff = abs(orig_topk[5] - adv_topk[5])
    eps_emp = compute_empirical_eps(input_imgs, adv_imgs_fgsm)
    fgsm_score, tp1_s, tp5_s = estimate_fgsm_score(tp1_diff, tp5_diff, eps_emp)

    best_fgsm_eps = eps
    best_fgsm_adv_top1  = adv_topk[1]
    best_fgsm_adv_top5  = adv_topk[5]
    best_fgsm_imgs = adv_imgs_fgsm
    best_fgsm_labels = adv_lbls_fgsm
    best_fgsm_eps_emp = eps_emp
    best_fgsm_score = fgsm_score

print("\n[FGSM] Chosen eps:", best_fgsm_eps)
print("[FGSM] Chosen eps_emp:", best_fgsm_eps_emp)
print("[FGSM] Adversarial Top-1 acc:", best_fgsm_adv_top1)
print("[FGSM] Adversarial Top-5 acc:", best_fgsm_adv_top5)
print("[FGSM] Estimated FGSM score (linear part):", best_fgsm_score)

# Save final best FGSM images as List[Tensor] only
assert best_fgsm_imgs is not None
assert len(best_fgsm_imgs) == NUM_SAMPLES
assert best_fgsm_imgs[0].shape == torch.Size([3, 32, 32])

torch.save(best_fgsm_imgs, "fgsm.pt")
print("Saved FGSM adversarial images to fgsm.pt")


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 39.16it/s]


[FGSM eps=0.050] Top1_adv=0.130, Top5_adv=0.750, tp1_diff=0.690, tp5_diff=0.220, eps_emp=0.049, tp1_score=23.100, tp5_score=14.520, est_score=10.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 19.61it/s]


[FGSM eps=0.080] Top1_adv=0.050, Top5_adv=0.610, tp1_diff=0.770, tp5_diff=0.360, eps_emp=0.079, tp1_score=25.740, tp5_score=23.760, est_score=10.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 41.27it/s]


...

[FGSM eps=0.160] Top1_adv=0.030, Top5_adv=0.490, tp1_diff=0.790, tp5_diff=0.480, eps_emp=0.155, tp1_score=26.400, tp5_score=31.680, est_score=10.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 46.01it/s]


[FGSM eps=0.180] Top1_adv=0.040, Top5_adv=0.490, tp1_diff=0.780, tp5_diff=0.480, eps_emp=0.174, tp1_score=26.070, tp5_score=31.680, est_score=10.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 46.70it/s]

[FGSM eps=0.200] Top1_adv=0.050, Top5_adv=0.520, tp1_diff=0.770, tp5_diff=0.450, eps_emp=0.193, tp1_score=25.740, tp5_score=29.700, est_score=10.000

[FGSM] Chosen eps: 0.05
[FGSM] Chosen eps_emp: 0.04933002591133118
[FGSM] Adversarial Top-1 acc: 0.13
[FGSM] Adversarial Top-5 acc: 0.75
[FGSM] Estimated FGSM score (linear part): 10.0
Saved FGSM adversarial images to fgsm.pt


In [11]:

pgd_eps_grid = [0.05, 0.10, 0.15, 0.20, 0.25]
PGD_ITERS = 40

best_pgd_eps = None
best_pgd_adv_top1 = None
best_pgd_adv_top5 = None
best_pgd_imgs = None
best_pgd_labels = None

for eps in pgd_eps_grid:
    # step size: relatively aggressive but still within eps/iters
    alpha = eps / PGD_ITERS

    adv_imgs_pgd, adv_lbls_pgd = build_adv_dataset_pgd(
        model, testloader, epsilon=eps, alpha=alpha, num_iter=PGD_ITERS
    )
    adv_topk = compute_adv_topk(model, adv_imgs_pgd, adv_lbls_pgd, batch_size=BATCH_SIZE)

    tp1_diff = abs(orig_topk[1] - adv_topk[1])
    tp5_diff = abs(orig_topk[5] - adv_topk[5])
    eps_emp = compute_empirical_eps(input_imgs, adv_imgs_pgd)

    pgd_bonus, tp1_s, tp5_s = estimate_pgd_score(tp1_diff, tp5_diff, eps_emp)

    print(
        f"[PGD eps={eps:.3f}] "
        f"Top1_adv={adv_topk[1]:.3f}, Top5_adv={adv_topk[5]:.3f}, "
        f"tp1_diff={tp1_diff:.3f}, tp5_diff={tp5_diff:.3f}, eps_emp={eps_emp:.3f}, "
        f"tp1_score={tp1_s:.3f}, tp5_score={tp5_s:.3f}, est_bonus={pgd_bonus:.3f}"
    )

    # Selection rule: minimize adv Top-5 accuracy, then adv Top-1
    if best_pgd_eps is None:
        choose = True
    else:
        choose = (
            adv_topk[5] < best_pgd_adv_top5 or
            (math.isclose(adv_topk[5], best_pgd_adv_top5) and adv_topk[1] < best_pgd_adv_top1)
        )

    if choose:
        best_pgd_eps = eps
        best_pgd_adv_top1 = adv_topk[1]
        best_pgd_adv_top5 = adv_topk[5]
        best_pgd_imgs = adv_imgs_pgd
        best_pgd_labels = adv_lbls_pgd

print("\n[PGD] Best eps:", best_pgd_eps)
print("[PGD] Best adversarial Top-1 acc:", best_pgd_adv_top1)
print("[PGD] Best adversarial Top-5 acc:", best_pgd_adv_top5)

assert len(best_pgd_imgs) == NUM_SAMPLES
assert best_pgd_imgs[0].shape == torch.Size([3, 32, 32])

torch.save(best_pgd_imgs, "pgd.pt")
print("Saved PGD adversarial images to pgd.pt")


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 45.13it/s]


[PGD eps=0.050] Top1_adv=0.030, Top5_adv=0.580, tp1_diff=0.790, tp5_diff=0.390, eps_emp=0.028, tp1_score=20.540, tp5_score=17.940, est_bonus=2.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 35.23it/s]


[PGD eps=0.100] Top1_adv=0.000, Top5_adv=0.380, tp1_diff=0.820, tp5_diff=0.590, eps_emp=0.045, tp1_score=21.320, tp5_score=27.140, est_bonus=2.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 42.48it/s]


[PGD eps=0.150] Top1_adv=0.000, Top5_adv=0.270, tp1_diff=0.820, tp5_diff=0.700, eps_emp=0.056, tp1_score=21.320, tp5_score=32.200, est_bonus=2.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 32.75it/s]


[PGD eps=0.200] Top1_adv=0.000, Top5_adv=0.220, tp1_diff=0.820, tp5_diff=0.750, eps_emp=0.065, tp1_score=21.320, tp5_score=34.500, est_bonus=2.000


Evaluating (clean)...: 100%|██████████| 13/13 [00:00<00:00, 24.53it/s]

[PGD eps=0.250] Top1_adv=0.000, Top5_adv=0.220, tp1_diff=0.820, tp5_diff=0.750, eps_emp=0.073, tp1_score=21.320, tp5_score=34.500, est_bonus=2.000

[PGD] Best eps: 0.2
[PGD] Best adversarial Top-1 acc: 0.0
[PGD] Best adversarial Top-5 acc: 0.22
Saved PGD adversarial images to pgd.pt
